In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2
from scipy.interpolate import interp1d
import os
import sys
sys.path.append('/home/astridaurora/HESE-7-year-data-release/HESE-7-year-data-release')
from Astrid.effective_area import bin_edges_to_centers, bin_centers_to_edges, apply_energy_smearing

import nuSIprop

This code is for larger datasets where one can use the Chi2 computations for Gaussian distributions with larger n, or when one wants to avoid the x_i=zero numerical divergence by grouping the events. 

In [ ]:
def compute_chi2(observed, expected, err=None):
    """
    Compute the Chi-squared statistic between two DataFrames.
    
    Parameters:
    observed : array-like
        Observed values.
    expected : array-like
        Expected values. Must have the same shape as observed.
    
    Returns:
    chi2_value : float
        The chi-squared value.
    p_value : float
        The p-value corresponding to the chi-squared test.
    """ 
    # Ensure DataFrames have the same shape
    if observed.shape != expected.shape:
        raise ValueError("Arrays must have the same shape")

    # Degrees of freedom = number of rows - 1
    dof = len(observed) - 1

    zeros_mask = (observed != 0)
    if observed.any() == 0:
        print('observed data contains zeros')
    

    # Compute chi-squared statistic
    if err is None:
        chi2_value = np.sum((observed[zeros_mask] - expected[zeros_mask]) ** 2 / observed[zeros_mask])
    else:
        chi2_value = np.sum((observed[zeros_mask] - expected[zeros_mask]) ** 2 / err[zeros_mask]**2)

    # Calculate p-value
    p_value = chi2.sf(chi2_value, dof)
    
    return chi2_value, p_value

In [ ]:
def group_events(df):
    """
    Group specified indices in the HESE-12 dataset and update energy centers.
    Groups (0,1), (14,15), and (16,17,18,19) respectively.
    Uses geometric mean for energy centers since bins are in logspace.
    """
    # Create a copy of the dataframe
    grouped_df = df.copy()
    # Create a mask for the indices we want to keep
    zeros_index = [0, 14, 16, 18, 19]
    
    # Energy centers for the grouped bins
    energy_center_01 = (df.index[0] * df.index[1]) ** 0.5
    energy_center_1415 = (df.index[14] * df.index[15]) ** 0.5
    energy_center_1619 = (df.index[16] * df.index[17] * df.index[18] * df.index[19]) ** 0.25
    
    # Events for the grouped bins
    events_01 = grouped_df.iloc[0]['events'] + grouped_df.iloc[1]['events']
    events_1415 = grouped_df.iloc[14]['events'] + grouped_df.iloc[15]['events']
    events_1619 = grouped_df.iloc[16]['events'] + grouped_df.iloc[17]['events'] + grouped_df.iloc[18]['events'] + grouped_df.iloc[19]['events']
    #print(events_01, events_1415, events_1619)
    # Update the dataframe
    index = grouped_df.index.values
    index[1] = energy_center_01
    index[15] = energy_center_1415
    index[17] = energy_center_1619
    index = np.delete(index, zeros_index)  
    events = grouped_df['events'].values
    events[1] = events_01
    events[15] = events_1415
    events[17] = events_1619
    events = np.delete(events, zeros_index)
    
    grouped_df_new = pd.DataFrame({'events': events}, index=index)
    #print(grouped_df_new)

    return grouped_df_new


hese12_grouped_df = group_events(hese12_events_df)
background_grouped_df = group_events(background_df)

# Subtract the background events from the HESE12 data, so that if the background is large, it will show in the chi2.
observed_signal = hese12_grouped_df['events'].values - background_grouped_df['events'].values
print(observed_signal)
print('total observed signal events', observed_signal.sum())

In [ ]:
g_phi = np.logspace(-4, 0, num=10)
M_phi = np.logspace(np.log10(4*1e-1), np.log10(2*1e2), num=10)
chi2_grid = np.zeros(shape=(len(g_phi), len(M_phi)))

evolver = nuSIprop.pyprop(mphi = M_phi[0]*1e6, # Mediator mass [eV]
			  g = g_phi[0], # Coupling
			  mntot = 0.1, # Sum of neutrino masses [eV]
			  si = si_grid[0], # Spectral index
			  norm = norm, # Normalization of the free-streaming flux at 100 TeV [Default = 1]
			  majorana = True, # Majorana neutrinos? [Default = True]
			  non_resonant = True, # Include non s-channel contributions? Relevant for couplings g>~0.1 [Default = True]
			  normal_ordering = True, # Normal neutrino mass ordering? [Default = True]
			  N_bins_E = 300, # Number of energy bins, uniformly distributed in log space [Default = 300]
			  lEmin = 13, # log_10 (E_min/eV) [Default = 13]
			  lEmax = 16, # log_10 (E_max/eV) [Default = 17]
			  zmax = 5, # Largest redshift at which sources are included [Default = 5]
			  flav = 2, # Flavor of interacting neutrinos [0=e, 1=mu, 2=tau. Default = 2]
			  phiphi = False # Consider double-scalar production? If set to true, the files xsec/alpha_phiphi.bin and xsec/alphatilde_phiphi.bin must exist [Default = False]
                          )
evolver.evolve()
flx = evolver.get_flux_fla()
energies = evolver.get_energies()
bin_edges_high_resolution = bin_centers_to_edges(energies)
delta_E = np.diff(bin_edges_high_resolution)    # Delta E in eV

In [ ]:
for g_idx, g in enumerate(g_phi):
    for M_idx, M in enumerate(M_phi):
        chi2_si = []
        for si_idx, si_ in enumerate(si_grid):
            #print(f'g = {g}, M = {M}, si = {si_}')

            with SuppressOutput():
                evolver.set_parameters(g=g, mphi=M*1e6, si=si_)
                evolver.evolve()

            flx = evolver.get_flux_fla()
            flx_df = pd.DataFrame(flx.T, index=evolver.get_energies(), columns=['nu_e', 'nu_mu', 'nu_tau'])
            flx_df.index = flx_df.index / 1e9    # Convert to [GeV] to align with eff_df. 

            # norm = 1e-4 to account for cm to m conversion
            nuSIprop_df = total_events(flx=effective_area_df, eff=flx_df, livetime=livetime12, norm=1e-4, delta_E=delta_E)
            nuSIprop_events = nuSIprop_df['total_events']
            nuSIprop_smeared = apply_energy_smearing(energies=nuSIprop_df.index.values, events=nuSIprop_events.values, resolution=0.1)
            
            # Bin the nuSIprop events according to the low resolution energy bins, to be compared with the HESE12 data.
            # Group the nuSIprop events manually, as equal to the HESE12 data
            # to avoid numerical issues from zeros in chi2 calculation.
            nuSIprop_binned_events, _ = np.histogram(a=nuSIprop_df.index.values, weights=nuSIprop_events.values, bins=energy_bins_low_resolution)
            nuSIprop_binned_df = pd.DataFrame(nuSIprop_binned_events, index=energy_centers_low_resolution, columns=['events'])
            nuSIprop_grouped_df = group_events(nuSIprop_binned_df)
            assert np.round(nuSIprop_grouped_df['events'].sum(), 10) == np.round(nuSIprop_binned_df['events'].sum(), 10)
            #print(nuSIprop_grouped_df['events'].values)
            
            # Add the background events to the nuSIprop events. 
            #predicted = nuSIprop_grouped_df['events'].values + background_grouped_df['events'].values
            #print('total predicted events', predicted.sum())

            # Compute the chi2 value for the given g, M and si.
            chi2_, p_value = compute_chi2(observed=observed_signal, expected=nuSIprop_grouped_df['events'].values)
            chi2_si.append(chi2_)

        print('chi2_si', chi2_si)
        chi2_grid[g_idx, M_idx] = np.min(chi2_si)
        si_min = si_grid[np.argmin(chi2_si)]
        print(f'Minimum si for g={g}, M={M}: si = {si_min}')
        print('chi2_grid', chi2_grid)
        si_marginalized.append(si_min)